<a href="https://colab.research.google.com/github/leonardo3108/IA368dd/blob/main/exercicios/Aula_11/Aula_11_Aggregation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [2]:
!pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 15.3 MB/s eta 0:00:00


In [19]:
import os
import openai
import pandas as pd

In [22]:
openai.api_key = 'API_KEY_OPENAI'

# Obtenção da saída do Retriever

In [5]:
df = pd.read_csv('retriever.csv')
df

,question,text,rerank,docid,answer
0,Did the same team win the cup finals Watford p...,The cup has been won by the same team in two o...,1,1601617,no
1,Did the same team win the cup finals Watford p...,The victory meant Southampton reached the semi...,2,26877,no
2,Did the same team win the cup finals Watford p...,In the quarter finals they played fellow Premi...,3,516756,no
3,Did the same team win the cup finals Watford p...,The match was the fourth time that the two tea...,4,516746,no
4,Did the same team win the cup finals Watford p...,"The cup is currently held by Manchester City, ...",5,1601618,no
...,...,...,...,...,...
495,Who was the founder of Bell Aircraft?,"On February 23, 1909, Bell was present as the ...",6,2416112,Larry Bell
496,Who was the founder of Bell Aircraft?,Burford has twice had a bell foundry: one run ...,7,1722969,Larry Bell
497,Who was the founder of Bell Aircraft?,"Before long, Bell became general manager and b...",8,1782967,Larry Bell
498,Who was the founder of Bell Aircraft?,"The company was purchased in 1960 by Textron, ...",9,1782963,Larry Bell


# Preparação do prompt

In [26]:
def message_user(numExemplo, pergunta, lista_documentos):
  doc_com_id = [ f"[Document {i+1}]: {lista_documentos[i]}" for i in range(len(lista_documentos))]
  return f"Example {numExemplo}\n\n" + "\n\n".join(doc_com_id) + f"\n\nQuestion: {pergunta}"

def message_assistant(evidencia, resposta):
  return f"Evidence: {evidencia}\n\nAnswer: {resposta}"

few_shot_examples = [
    {"user": "", "assistant": ""},
    {"user": "", "assistant": ""},
]
few_shot_examples[0]["user"] = message_user(1, "Did Pink Floyd have a song about the French Riviera?", [
    "\"San Tropez\" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band's \"best of\" album, Echoes: The Best of Pink Floyd.",
    "The French Riviera (known in French as the Côte d'Azur; literal translation \"Azure Coast\") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d'Azur (Région Sud) region of France.",
    "Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.",
    "Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz , pronounced [san(t) tʀuˈpes]) is a town on the French Riviera, 68 kilometres (42 miles) west of Nice and 100 kilometres (62 miles) east of Marseille in the Var department of the Provence-Alpes-Côte d'Azur region of Occitania, Southern France."
])
few_shot_examples[0]["assistant"] = message_assistant("\"San Tropez\" is a song by Pink Floyd about the French Riviera [Document 1]. Saint-Tropez is a town on the French Riviera [Document 4]", "yes")
few_shot_examples[1]["user"] = message_user(2, "On what date did the Loma Prieta earthquake occur?", [
    "The 1989 Loma Prieta earthquake occurred on California’s Central Coast on October 17 at local time (1989-10-18 00:04 UTC). The shock was centered in The Forest of Nisene Marks State Park approximately 10 mi northeast of Santa Cruz on a section of the San Andreas Fault System and was named for the nearby Loma Prieta Peak in the Santa Cruz Mountains. With an magnitude of 6.9, the shock was responsible for 63 deaths and 3,757 injuries.",
    "'Foreshocks. The 5.3 June 1988 and the 5.4 August 1989 events also occurred on previously unknown oblique reverse faults and were within 3 mi of the M6.9 Loma Prieta mainshock epicenter, near the intersection of the San Andreas and Sargent faults. Total displacement for these shocks was relatively small.'",
    "The June 27, 1988, shock occurred with a maximum intensity of VI (Strong). Its effects included broken windows in Los Gatos, and other light damage in Holy City, where increased flow was observed at a water well. Farther away from the Santa Cruz Mountains, pieces of concrete fell from a parking structure at the Sunnyvale Town Center, a two-level shopping mall in Santa Clara County.",
    "The 6.9 Loma Prieta earthquake occurred on October 17, 1989. The rupture was related to the San Andreas fault system and affected the entire San Francisco Bay Area with a maximum Mercalli intensity of IX (Violent). Many structures in Oakland were badly damaged including the double-decker portion of Interstate 880 that collapsed.",
])
few_shot_examples[1]["assistant"] = message_assistant("According to [Document 1] and [Document 4], the Loma Prieta earthquake occur on October 17, 1989", "1989-10-18")
few_shot_examples

[{'user': 'Example 1\n\n[Document 1]: "San Tropez" is the fourth track from the album Meddle by the band Pink Floyd. This song was one of several to be considered for the band\'s "best of" album, Echoes: The Best of Pink Floyd.\n\n[Document 2]: The French Riviera (known in French as the Côte d\'Azur; literal translation "Azure Coast") is the Mediterranean coastline of the southeast corner of France. There is no official boundary, but it is usually considered to extend from Cassis, Toulon or Saint-Tropez on the west to Menton at the France–Italy border in the east, where the Italian Riviera joins. The coast is entirely within the Provence-Alpes-Côte d\'Azur (Région Sud) region of France.\n\n[Document 3]: Moon also promised transparency in his presidency, moving the presidential residence from the palatial and isolated Blue House to an existing government complex in downtown Seoul.\n\n[Document 4]: Saint-Tropez (US: /ˌsæn troʊˈpeɪ/ SAN-troh-PAY, French: [sɛ̃ tʁɔpe]; Occitan: Sant-Tropetz

# Execução

In [ ]:
result = []

for question_idx in range(50):
    print(f'Processing question #{question_idx + 1}...')
    question = df.loc[question_idx * 10, 'question']
    expected = df.loc[question_idx * 10, 'answer']
    texts = df.loc[question_idx * 10 : question_idx * 10 + 10, 'text'].tolist()
    response = openai.ChatCompletion.create(model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful reader that reads a list of documents and use them as evidence to answer a question. You provide detailed explanations and concise answers."},
            {"role": "user", "content": "For each example, write an explanation to the answer given the documents and the question. The final answer should be short (less than 10 words)."},
            {"role": "user", "content": few_shot_examples[0]["user"]},
            {"role": "assistant", "content": few_shot_examples[0]["assistant"]},
            {"role": "user", "content": few_shot_examples[1]["user"]},
            {"role": "assistant", "content": few_shot_examples[1]["assistant"]},
            {"role": "user", "content": message_user(2, question, texts)}
        ],
        temperature=0,
        max_tokens=100)
    result.append({'question': question, 'answer': response['choices'][0]['message']['content'], 'expected': expected, 'texts': texts})

# Avaliação

In [66]:
df_result = pd.DataFrame(result)
del df_result['texts']
df_result

,question,answer,expected
0,Did the same team win the cup finals Watford p...,Evidence: Manchester City won the 2019 FA Cup ...,no
1,How deep is the body of water where the Siberi...,Evidence: The Laptev Sea is located between th...,1642 m
2,How did the Fourth Brigade earn its title in t...,"Evidence: The Fourth Brigade was called the ""I...",connotation of fighting men with iron disposit...
3,How long did the emperor serve that granted Bo...,Evidence: [Document 1] states that Emperor Hen...,6
4,How long did the monarch who favored Alice Lei...,Evidence: Victoria was the longest-reigning Br...,45
5,How long had the Warren Commission been establ...,Evidence: There is no document that mentions O...,1
6,How long was the reign of the king that Pope C...,Evidence: There is no mention of a king's reig...,49
7,How many NCAA football championships have been...,Evidence: There is no mention of Shires' alma ...,16
8,How many people live in the city that hosts KOGO?,Evidence: There is no clear evidence in the do...,"1,425,976"
9,How many songs are on Blind Guardian's concept...,Evidence: Nightfall in Middle-Earth is a conce...,25


In [67]:
# Funções normize_answer e token_f1_score foram retiradas do Visconde (arquivo qasper_evaluator.py)
import re
import string
from collections import Counter

def normalize_answer(s):
    """
    Taken from the official evaluation script for v1.1 of the SQuAD dataset.
    Lower text and remove punctuation, articles and extra whitespace.
    """

    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)

    def white_space_fix(text):
        return " ".join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def token_f1_score(prediction, ground_truth):
    """
    Taken from the official evaluation script for v1.1 of the SQuAD dataset.
    """
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1

def extract_short_answer(answer):
    position = answer.find('Answer: ') + 8
    return answer[position:]

# Trata respostas vazias do GPT-3.5
def standardize_answer(answer):
    if answer == '':
        return 'none'

    answer = answer.lower()
    void_answers = ["n/a", "it is not possible to determine", "no information in the documents", "no information provided", "none of the documents", "documents do not provide", "unknown", "cannot be answered", "not mentioned", "not specified", "n/a", "not available", "insufficient data", "not enough information", "cannot be determined", "there is no mention"]
    for void_answer in void_answers:
        if void_answer in answer:
            return 'none'
    return answer

In [68]:
df_result['normalized_answer'] = ''
df_result['normalized_expected'] = ''
df_result['F1_score'] = ''

In [69]:
for line in df_result.index:
    question = df_result.loc[line, 'question']
    answer = normalize_answer(standardize_answer(extract_short_answer(df_result.loc[line, 'answer'])))
    expected = normalize_answer(df_result.loc[line, 'expected'])
    df_result.loc[line, 'normalized_answer'] = answer
    df_result.loc[line, 'normalized_expected'] = expected
    df_result.loc[line, 'F1_score'] = token_f1_score(answer, expected)
df_result

,question,answer,expected,normalized_answer,normalized_expected,F1_score
0,Did the same team win the cup finals Watford p...,Evidence: Manchester City won the 2019 FA Cup ...,no,no,no,1.0
1,How deep is the body of water where the Siberi...,Evidence: The Laptev Sea is located between th...,1642 m,none,1642 m,0
2,How did the Fourth Brigade earn its title in t...,"Evidence: The Fourth Brigade was called the ""I...",connotation of fighting men with iron disposit...,brave fighting,connotation of fighting men with iron disposit...,0.222222
3,How long did the emperor serve that granted Bo...,Evidence: [Document 1] states that Emperor Hen...,6,2 years,6,0
4,How long did the monarch who favored Alice Lei...,Evidence: Victoria was the longest-reigning Br...,45,none,45,0
5,How long had the Warren Commission been establ...,Evidence: There is no document that mentions O...,1,none,1,0
6,How long was the reign of the king that Pope C...,Evidence: There is no mention of a king's reig...,49,none,49,0
7,How many NCAA football championships have been...,Evidence: There is no mention of Shires' alma ...,16,none,16,0
8,How many people live in the city that hosts KOGO?,Evidence: There is no clear evidence in the do...,"1,425,976",none,1425976,0
9,How many songs are on Blind Guardian's concept...,Evidence: Nightfall in Middle-Earth is a conce...,25,none,25,0


# Resultado

In [73]:
df_result['F1_score'].mean()

0.1417171717171717